## 1) Imports

In [1]:
# libraries
import os
import toml
import utils as util_fun
import concurrent.futures

# modules
import tracking_models as track

## 2) Config

In [2]:
# read config data
with open('config.toml', 'r') as f:
    config = toml.load(f)

# Hand and Pose landmark naming
hands_name_lst: list = config['body_parts']['hands_landmark_lst']
pose_name_lst: list = config['body_parts']['pose_landmark_lst']

# Hand and Pose tracking model path
tracking_model_path: str = config['body_parts']['tracking_model_path']
hand_model_path: str = os.path.join(tracking_model_path, 'hand_landmarker.task')
pose_model_path: str = os.path.join(tracking_model_path, 'pose_landmarker_heavy.task')

## 3) Run Batch Tracking with Parallel Processes

In [ ]:
# handles the entire tracking and saving process for one video file.
def process_single_video(video_task):
    """
    Worker function to process a single video for hand and/or pose tracking.

    Args:
        video_task (list): list of video path and video name

    Returns:
        None
    """

    # get info from video task
    vid_path, out_path, out_path_lst, tracking_selection = video_task

    # Hand tracking
    if tracking_selection == 1:
        video_name_hands: str = (os.path.basename(vid_path)).split('.')[0] + '_hands.csv'
        if video_name_hands not in out_path_lst:
            # apply hands tracking model
            hands_data_dict: dict = track.hand_landmark_extractor(vid_path, out_path, hand_model_path, n_hands=2,
                                                                  min_hand_detect_conf=0.5, min_hand_track_conf=0.8,
                                                                  normalize=True, visualize=False, save_video=True)
            # save marker data to csv
            util_fun.save_hands_to_csv(video_name_hands, out_path, hands_data_dict, hands_name_lst)
            return f'Finished Hand Tracking for {os.path.basename(vid_path)}'

    # Pose tracking
    elif tracking_selection == 2:
        video_name_pose: str = (os.path.basename(vid_path)).split('.')[0] + '_pose.csv'
        if video_name_pose not in out_path_lst:
            # apply pose tracking model
            pose_data_dict: dict = track.pose_landmark_extractor(vid_path, out_path, pose_model_path,
                                                                 min_pose_detect_conf=0.5, min_pose_track_conf=0.8,
                                                                 normalize=True, visualize=False, save_video=True)
            # save marker data to csv
            util_fun.save_pose_to_csv(video_name_pose, out_path, pose_data_dict, pose_name_lst)
            return f'Finished Pose Tracking (UL) for {os.path.basename(vid_path)}'

    return None

tracking_base_path: str = config['batch_tracking']['tracking_base_path']
tracking_src_path: str = config['batch_tracking']['tracking_src_path']
tracking_out_path: str = config['batch_tracking']['tracking_out_path']
participant_fname_lst: list = [x for x in sorted(os.listdir(tracking_src_path)) if x.startswith('P')]

# list with all videos of exercises from all participants
all_tasks = []

# process for each participant
for participant_fname in participant_fname_lst:

    # update source and result paths
    src_fpath = os.path.join(tracking_src_path, participant_fname)
    res_fpath = os.path.join(tracking_out_path, participant_fname)

    # create a list of video file paths
    video_path_lst: list[str] = util_fun.import_video_files(src_fpath)

    # create a directory for the resulting files (skip if already existing)
    os.makedirs(res_fpath, exist_ok=True)

    # create a list of already existing result files
    res_path_lst: list[str] = [x.split('.')[0] for x in os.listdir(res_fpath) if x.endswith('.csv')] if os.listdir(res_fpath) else []

    print(f'Queueing videos of {participant_fname}...')

    # create tasks for both (1) Hand and (2) Pose tracking
    for video_path in video_path_lst:
        # (1) task for Hand tracking
        all_tasks.append((video_path, res_fpath, res_path_lst, 1))
        # (2) task for Pose Tracking
        all_tasks.append((video_path, res_fpath, res_path_lst, 2))

# define multiple workers for parallel execution
MAX_WORKERS = config['batch_tracking']['max_workers']

print(f'\nStarting parallel processing of {len(all_tasks)} tasks with {MAX_WORKERS} workers...')

with concurrent.futures.ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # Distribute tasks to the workers
    results = executor.map(process_single_video, all_tasks)

    # Print results as they complete
    for result in results:
        if result:
            print(f'Done: {result}')

print('\n --------- All tracking complete! ---------')